# An overview of OED with brmp

The describes the interface provided for performing model-selection driven OED. Also see `example.py` in this directory for an interactive command line based example.

In [1]:
from brmp.design import RealValued, Categorical, Integral
from brmp.oed import SequentialOED
from brmp.model import model_repr

## Setting up an OED problem

The main interface to the model selection driven OED is provided by the `SequentialOED` class. To set-up a new OED problem we first create an instance of this class. The main thing we need to provide here is a description of the model in the form of a lme4-like formula and metadata describing the columns mentioned in the formula.

In [2]:
metadata = [
    RealValued('y'),
    Categorical('a', levels=['a1', 'a2']),
    Categorical('b', levels=['b1', 'b2']),
]
oed = SequentialOED('y ~ 1 + a + b', metadata)

A `SequentialOED` instance carries a few useful properties. First, we can inspect the brmp model used internally by OED:

In [3]:
print(model_repr(oed.model_desc))  # TODO: Make this less clumsy.

Population
----------------------------------------
Coef Priors:
intercept       | Cauchy(loc=0.0, scale=1.0)
a[a2]           | Cauchy(loc=0.0, scale=1.0)
b[b2]           | Cauchy(loc=0.0, scale=1.0)
Response
----------------------------------------
Family: Normal()
Link:
  Parameter: mu
  Function:  identity
Priors:
sigma           | HalfCauchy(scale=3.0)


We can also access the data-so-far, which is represented as a Pandas data frame. This is empty initially.

In [4]:
oed.data_so_far

,y,a,b


By default, the OED will attempt to gain information about all coefficients appearing on the RHS of the formula.

In [5]:
oed.target_coefs

['b_intercept', 'b_a[a2]', 'b_b[b2]']

This default can be overridden by passing a list of coefficient names to the `SequentialOED` constructor.

In [6]:
SequentialOED('y ~ 1 + a + b', metadata, target_coefs=['b_a[a2]']).target_coefs

['b_a[a2]']

## Estimating the optimal next trial

To estimate an optimial next trial given the data-so-far, we call the `next_trial()` method on the `SequentialOED` instance.

In [7]:
design, _, eigs, fit, _ = oed.next_trial()

Targets interval:
  Low:  -0.5
  High: 0.5
Targets class balance: tensor([0.2950, 0.3030, 0.2910])


This method returns several values. Of most interest are:

* `design`, which is a dictionary containing a setting for each column on the RHS of the formula representing the estimated optimal next design.
* `eigs`, which is a list of all designs considered and their estimated (unnormalized) EIG.
* `fit`, which is the brmp `Fit` instance representing the posterior given the data-so-far.

In [8]:
design

{'a': 'a1', 'b': 'b2'}

In [9]:
eigs

[({'a': 'a1', 'b': 'b1'}, -1.7878224849700928),
 ({'a': 'a1', 'b': 'b2'}, -1.7712924480438232),
 ({'a': 'a2', 'b': 'b1'}, -1.7863678932189941),
 ({'a': 'a2', 'b': 'b2'}, -1.7882044315338135)]

In [10]:
fit.marginals()

              mean      sd   2.5%   25%   50%   75%  97.5%    n_eff r_hat
b_intercept  -0.62   17.43 -12.28 -0.98  0.01  1.04  11.48   993.06  1.00
    b_a[a2]  -0.86  121.85 -11.47 -0.86  0.04  1.11  12.51   989.78  1.00
    b_b[b2]  12.53  403.29 -11.12 -0.97 -0.00  1.00  16.11  1002.28  1.00
      sigma  13.70   91.51   0.14  1.17  2.92  7.31  71.99  1017.24  1.00

By default an exhaustive search is performed over the full Cartesian product of the levels(/possible values) taken on by each of the columns mentioned in the RHS of the formula. The search can be restricted to a subset of this by passing a `design_space` argument to `next_trial()`.

In [11]:
design_space = [{'a': 'a2', 'b': 'b1'}, {'a': 'a2', 'b': 'b2'}]
_, _, eigs2, _, _ = oed.next_trial(design_space=design_space)
eigs2

Targets interval:
  Low:  -0.5
  High: 0.5
Targets class balance: tensor([0.2880, 0.3180, 0.3110])


[({'a': 'a2', 'b': 'b1'}, -1.811561107635498),
 ({'a': 'a2', 'b': 'b2'}, -1.8157528638839722)]

## Adding results

Once a trial has been run, the result can be added to the data-so-far using the `add_result` method. This takes two arguments -- the design that was run (using the same representation of a trial the `next_trial` returns) and the result itself.

In [12]:
oed.add_result(design, -2.0)
oed.data_so_far

,y,a,b
0,-2.0,a1,b2
